In [1]:
%cd ../..

/home/pristalovya/Документы/nlp-coursework


In [2]:
from src.nlp.application import Pipeline, get_df_by_film_and_person, get_df_by_person, collect_sents_to_summarize
from datasets_ import DatasetLoader

import dill
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib notebook


from transformers import MBartTokenizer, MBartForConditionalGeneration
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/home/pristalovya/Inter/linux_packages/anaconda3/envs/nlp/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
data = DatasetLoader.load_named_entities_dataset(show_path=True)
data

/home/pristalovya/Документы/nlp-coursework/data/named_entities.csv


,ne,occurrences,film_id,n_sents
0,Кинг,[Несколько лет назад прочитала оригинальный ро...,435,150
1,Дарабонт,[После взросления и прочтения книги фильм Дара...,435,65
2,Перси,[Особенно нелепа вся эта линия с травлей Перси...,435,93
3,Депрессия,"[За то, что работает по блату (во времена Депр...",435,1
4,Делакруа,"[И кстати, ради большей слезовыжималки сценари...",435,26
...,...,...,...,...
106669,Сергей Эйзенштейн,[В качестве наглядного примера могу предложить...,969760,1
106670,Ферруччо,[Молодой Ферруччо - типичный англо-саксонский ...,969760,9
106671,Эдуардо де Филиппо,[Авторам смотреть старые итальянские фильмы и ...,969760,1
106672,Ютюба,[И он по законам мотивашек с Ютюба читает как ...,969760,1


In [4]:
listed_opinions = collect_sents_to_summarize(get_df_by_person(data, 'стивен кинг'))
opinions = '\n'.join(listed_opinions)

In [5]:
opinions

"Зачем тратить столько сил, качественно выполнять свою работу и стараться, но при этом делать это по абсолютно бредовому сценарию, гордо ссылающемуся на произведение Стивена Кинга, от которого в нём мало что осталось, кроме названий, имён и статуи Пола Баньяна?\nПомимо раскрытия героев, Стивен Кинг сделал клоуна Пеннивайза олицетворением кошмаров детей и взрослых, в литературных рамках уничтожив образ доброго клоуна так, что в нем не осталось ничего, кроме всепоглощающего ужаса.\nНовый фильм, не шедевр, но как вполне себе неплохой ужастик заслуживает оценки \r\n\r\n6 из 10\r\n\r\n Ну, а как, глубокий философский фильм, пугающий своей атмосферой и соответствующий замыслу оригинала Стивена Кинга – 4 из 10.\nОтправляясь в кино на экранизацию произведения своего любимого Стивена Кинга я была  в предвкушении увидеть если не свои образы от прочтения книги, то хотя бы эволюционировавшую версию 'ОНО' 1990 года Томми Ли Уоллеса, но, к сожалению, была немного разочарована.\nОт того и многочислен

In [5]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

input_ids = tokenizer(
    [opinions],
    max_length=600,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4,
    min_length=500,
    max_length=1000
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)


В российском прокате «Зелёная миля» — экранизация романа Стивена Кинга «Зеленая Миля», снятая по одноимённому роману. Сюжет схож с «Мастером и Маргаритой» Стивена Кинга, поэтому я не буду называть эту картину «королем ужасов» и скажу, что она стала для меня одной из самых любимых экранизаций в кинематографе, несмотря на то, что я не читала ее до сих пор, а просто смотрела фильм «Стивен Кинг» с Андреем Толстым в главной роли, который мне понравился больше, чем «Мастер и Маргарита» от Кинга, а также «А. Толстой» от Стивена. Скотта и «Оклахома» Уиллисасасаса в главной роли в фильме «Стивенабиль-Большенье» в исполнении Стивена Стивена Бертона-Сартандарта в исполнении Стивена. Стивен Кинга и Фрэнка Дарабона в главной роли Толстяса в картине Кинга в главной роли Стивена-Стивена в «Зелевадера в исполнении Кинга в «Зеленой Миле.» и «Зелеваяготанца.» Стивена в главной ролика в картине «Зеленой «Зеля в «Зеленогойзера» и «Зеленой» «Зеленой миля» «Зе» «Зеленого» Кинга «Зелеса «Зе.» «Зеленая» «Зеле

In [5]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

for opinion in listed_opinions[::-1]:

    input_ids = tokenizer(
        [opinion],
        max_length=600,
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4,
        max_new_tokens=None
    )[0]

    print(tokenizer.decode(output_ids, skip_special_tokens=True), end='\n\n')


/home/pristalovya/Inter/linux_packages/anaconda3/envs/nlp/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Алиса Кингсли, Миа Васиковска и Алиса Бартона, Алиса Киингсли и Тим Бартон - что посмотреть в кино в эти выходные.

Актриса Алиса Кингсли вновь взялась за свою прошлую роль в роли независимой женщины. Миа Васиковска снова взяла за свой прошлой роль, предоставив нам новый образ 'независимой женщиной'.

Миа Васиковска возвращается из очередного морского путешествия Алиса Кингсли и узнает, что бизнес её на грани закрытия, а будущее не сулит ничего хорошего.

Алиса Кингсли оказалась в беде за три года отсутствия дома, однако ее близкие вновь оказались в беде, как в Лондоне, так и в Стране Чудес.

Алиса Кингсли явилась не в образе спасительницы, а в образе живой туши.

Алиса Алиса Кингсли заметно заметно взрослела Алиса Кинсли заметно старше Алисы Кингсли, а Алиса Кингу заметно выглядела взрослой и заметно вывзрослевшей Алису Кингсли. Алиса уже заметно выросла заметно и взрослела, и Алиса будет взрослеть до взрослого возраста Алиса Киингсли.

В Лондоне начался показ сериала «Салли Локхарт»,

In [30]:
# listed_opinions = collect_sents_to_summarize(get_df_by_person(data, 'камбербетч'))
# opinions = '\n'.join(listed_opinions)

# Load Packages
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

# For Strings
parser = PlaintextParser.from_string(opinions, Tokenizer("russian"))
from sumy.summarizers.text_rank import TextRankSummarizer

# Summarize using sumy TextRank
summarizer = TextRankSummarizer()
summary =summarizer(parser.document, 5)
text_summary=""

for sentence in summary:
    text_summary += str(sentence)
print(text_summary)

Клинт Иствуд сделал «Малышку на миллион» очень жёсткой и честной, он отошёл от привычных конвенций жанра про андердогов из отбросов общества и правдиво сказал, что вырваться из среды нищих практически невозможно, лучше смириться с этим и жить дальше, но внутри него, уборщика (Моргана Фримана) и Мэгги горит огонь ярости из-за того, что они несправедливо забыты обществом, что они никому не нужны.Для началу скажу, что про Клинта Иствуда я не хочу много говорить, этот человек давно уже всем доказал и продемонстрировал, что он очень хороший и актёр и режиссёр, очень хорошо удаётся ему  играть престарелых умудрённых опытом людей, которые в какой-то мере даже обижены на жизнь, в особенности в последних его работах, всё же возраст даёт о себе знать.Собственно Клинт Иствуд играет привычный для себя образ, неразговорчивого и грубого человека у которого практически нет друзей и который не любят, когда ему перечат, но вот только за внешней суровостью и безразличием скрывается доброе сердце и что с